In [1]:
import numpy as np
import time

def rosenbrock(x):
    x = np.asarray(x, float)
    return np.sum(100.0*(x[1:] - x[:-1]**2)**2 + (1.0 - x[:-1])**2)

def grad_rosenbrock(x):
    x = np.asarray(x, float)
    g = np.zeros_like(x)
    g[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    g[1:-1] = 200*(x[1:-1] - x[:-2]**2) - 400*x[1:-1]*(x[2:]-x[1:-1]**2) - 2*(1-x[1:-1])
    g[-1] = 200*(x[-1] - x[-2]**2)
    return g

def hess_rosenbrock(x):
    x = np.asarray(x, float)
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n-1):
        H[i, i] += 1200*x[i]**2 - 400*x[i+1] + 2
        H[i, i+1] = -400*x[i]
        H[i+1, i] = -400*x[i]
        H[i+1, i+1] += 200
    return H

def levenberg_marquardt(f, grad_f, hess_f, x0, tol=1e-6, lam=10.0, beta=2.0, max_iter=200, max_inner=50):
    x = np.asarray(x0, float).copy()
    I = np.eye(len(x))
    t0 = time.process_time()
    k = 0

    while k < max_iter and np.linalg.norm(grad_f(x)) > tol:
        fx = f(x)
        g = grad_f(x)
        H = hess_f(x)

        accepted = False
        for _ in range(max_inner):
            try:
                d = np.linalg.solve(H + lam*I, -g)
            except np.linalg.LinAlgError:
                lam *= beta
                continue

            x_new = x + d
            if f(x_new) < fx:
                x = x_new
                lam /= beta
                accepted = True
                break
            else:
                lam *= beta

        k += 1
        if not accepted:
            break

    t1 = time.process_time()
    print(f"CPU time: {t1 - t0:.6f} s")
    return x, f(x), k

n = 5
x0 = np.full(n, -1.2, float)
x0[1::2] = 1.0

x_star, f_star, iters = levenberg_marquardt(rosenbrock, grad_rosenbrock, hess_rosenbrock, x0)
print("Final point:", x_star)
print("Iterations:", iters)
print("Function value:", f_star)


CPU time: 0.000000 s
Final point: [-0.96205102  0.93573939  0.88071358  0.77787764  0.60509362]
Iterations: 18
Function value: 3.9308394341330293
